In [1]:
import sqlite3
!pip install names

## Part 1：Table Creation and Triggers

build connection to SQLite database, create tables based on the ER diagram, also create some triggers for the integrity purposes or business rules.

In [2]:
connection = sqlite3.connect("dating_app.db")

In [3]:
c = connection.cursor()

In [4]:
#establishing tables from the ER diagram

c.executescript('''           
CREATE TABLE `Report` (
  `report_id` INTEGER,
  `reporting_id` INTEGER,
  `reported_id` INTEGER,
  `time` DATE,
  `reason` VARCHAR,
    PRIMARY KEY (`report_id`)
);

CREATE TABLE `Profile` (
  `user_id` INTEGER,
  `name` VARCHAR,
  `sex` VARCHAR,
  `age` INTEGER,
  `status` VARCHAR,
  `height` INTEGER,
  `body_type` VARCHAR,
  `diet` VARCHAR,
  `drinks` VARCHAR,
  `drugs` VARCHAR,
  `smoke` VARCHAR,
  `ethnicity` VARCHAR,
  `education` VARCHAR,
  `job` VARCHAR,
  `location` VARCHAR,
  `pet` VARCHAR,
  `children` VARCHAR,
  `religion` VARCHAR,
  `languages` VARCHAR,
  `relationship_type` VARCHAR,
  `time` DATE,
  PRIMARY KEY (`user_id`)
);

CREATE TABLE `Membership_order` (
  `order_id` INTEGER PRIMARY KEY AUTOINCREMENT,
  `user_id` INTEGER,
  `order_date` DATE,
  `membership_duration` INTEGER,
  FOREIGN KEY (`user_id`) REFERENCES `Profile`(`user_id`)
);

CREATE TABLE `Preference` (
  `user_id` INTEGER,
  `sexual_orientation` VARCHAR,
  `age` INTEGER,
  `height` INTEGER,
  `body_type` VARCHAR,
  `diet` VARCHAR,
  `drinks` VARCHAR,
  `drugs` VARCHAR,
  `smoke` VARCHAR,
  `ethnicity` VARCHAR,
  `education` VARCHAR,
  `job` VARCHAR,
  `location` VARCHAR,
  `pet` VARCHAR,
  `children` VARCHAR,
  `religion` VARCHAR,
  `languanges` VARCHAR,
  `relationship_type` VARCHAR,
      FOREIGN KEY (`user_id`) REFERENCES `Profile`(`user_id`)
);

CREATE TABLE `Swipe` (
  `swiping_user_id` INTEGER,
  `swiped_user_id` INTEGER,
  `swipe_type` VARCHAR,
  `time` DATE,
  FOREIGN KEY (`swiping_user_id`) REFERENCES `Profile`(`user_id`),
  FOREIGN KEY (`swiped_user_id`) REFERENCES `Profile`(`user_id`)
);

CREATE TABLE `Match` (
  `match_id` INTEGER PRIMARY KEY AUTOINCREMENT,
  `user_id_1` INTEGER,
  `user_id_2` INTEGER,
  `time` DATE,
  FOREIGN KEY (`user_id_2`) REFERENCES `Profile`(`user_id`),
  FOREIGN KEY (`user_id_1`) REFERENCES `Profile`(`user_id`)
);

CREATE TABLE `Message` (
  `message_id` INTEGER PRIMARY KEY AUTOINCREMENT,
  `match_id` INTEGER,
  `sender_id` INTEGER,
  `receiver_id` INTEGER,
  `text` VARCHAR,
  `time` DATE,
  FOREIGN KEY (`match_id`) REFERENCES `Match`(`match_id`),
  FOREIGN KEY (`sender_id`) REFERENCES `Profile`(`user_id`),
  FOREIGN KEY (`receiver_id`) REFERENCES `Profile`(`user_id`)
);

CREATE TABLE `Block` (
  `block_id` INTEGER PRIMARY KEY AUTOINCREMENT,
  `blocking_id` INTEGER,
  `blocked_id` INTEGER,
  `time` DATE,
  FOREIGN KEY (`blocking_id`) REFERENCES `Profile`(`user_id`),
  FOREIGN KEY (`blocked_id`) REFERENCES `Profile`(`user_id`)
);


''')

In [5]:
#The Match trigger explained in report section 2.4 Business rules, constraints, triggers

#if the two users have swiped and liked each other
#then they will become a match and the data will be populated into the Match table

c.executescript('''CREATE TRIGGER matches AFTER INSERT ON Swipe
WHEN ((SELECT time FROM Swipe 
WHERE swiping_user_id=NEW.swiped_user_id AND swiped_user_id=NEW.swiping_user_id AND swipe_type='right' AND NEW.swipe_type='right'
)IS NOT NULL)
BEGIN
INSERT INTO Match(user_id_1,user_id_2,time) VALUES(NEW.swiped_user_id, NEW.swiping_user_id, NEW.time);
END;
''')

In [6]:
#The Block trigger explained in report section 2.4 Business rules, constraints, triggers

#The sender cannot send messages to receiver, if the receiver has blocked the sender

c.executescript('''CREATE TRIGGER blocks BEFORE INSERT ON Message
BEGIN
    SELECT CASE
    WHEN ((SELECT block_id FROM Block
            WHERE blocking_id=NEW.receiver_id AND blocked_id=NEW.sender_id 
            ) IS NOT NULL)
THEN RAISE(FAIL, 'ERROR: Message cannot be sent. The sender has been blocked by the receiver.')
END;
END;
''')

In [7]:
#This swipe_consistency trigger is not for the database design
#It is used for the convenience of data population

#because in the next part we use random.randint to insert into swiping and swiped user IDs, this may give repeated pairs
#This trigger ensures the swipe is unique and one user will not swipe the same user again

c.executescript('''CREATE TRIGGER swipe_consistency BEFORE INSERT ON Swipe
WHEN ((SELECT time FROM Swipe 
WHERE swiping_user_id=NEW.swiping_user_id AND swiped_user_id=NEW.swiped_user_id AND time<>NEW.time
)IS NOT NULL)
BEGIN
DELETE FROM Swipe 
WHERE swiping_user_id=NEW.swiping_user_id AND swiped_user_id=NEW.swiped_user_id;
END;
''')

## Part 2: Data Population (including synthetic data generation)
inserting data, reading the real-world dataset CSV from Kaggle, and generating synthetic data numpy random functions

In [8]:
import pandas as pd
import numpy as np
import names
from utils import *
import datetime

In [9]:
selected_columns = ['sex', 'age', 'status', 'height', 'body_type', 'diet', 'drinks',
                    'drugs', 'smokes', 'ethnicity', 'education', 'job', 'location', 'pets',
                    'offspring', 'religion', 'speaks', 'orientation']
idx_to_column = dict(enumerate(selected_columns))
column_to_idx = dict([(p, i) for (i, p) in idx_to_column.items()])
profiles = pd.read_csv("okcupid_profiles.csv")

#The CSV file we downloaded from Kaggle is too large so we did not upload it to Github
#But you can find it on
#https://www.kaggle.com/datasets/andrewmvd/okcupid-profiles

profiles = profiles[selected_columns]

In [10]:
#Because the profile file is too big, with around 55000 data, we only use 150 of them
#We are reading from the real Kaggle dataset

selected_idx = set(np.random.randint(0, 55000, (150,)))
selections = [np.array(profiles.iloc[idx]) for idx in selected_idx]
selections = np.array([[i, names.get_full_name()] + list(s) for (i, s) in enumerate(selections)])

In [11]:
persons = []
preferences = []
reports = []
memberships = []
swipes = []
messages = []
blocks = []

In [12]:
#Generating synthetic data, preparation

selected_columns = ['pid', 'name', 'sex', 'age', 'status', 'height', 'body_type', 'diet', 'drinks',
                    'drugs', 'smokes', 'ethnicity', 'education', 'job', 'location', 'pets',
                    'offspring', 'religion', 'speaks', 'orientation']
for i, selection in enumerate(selections):
    property = dict([(selected_columns[i], selection[i]) for i in range(len(selection))])
    property['relationship_type'] = np.random.choice(relationship_types)
    start = datetime.datetime(2010, 1, 1)
    end = datetime.datetime(2012, 12, 1)
    property['signup_time'] = str(random_date(start, end))

    p = Person(property)
    p.add_preference()
    preferences.append(p.get_preferences()[-1])
    if (i % 3 == 0):
        p.add_membership()
        memberships.append(p.get_memberships()[-1])

    persons.append(p)
    
for i in range(300):
    reporter = np.random.choice(persons)
    reportee = np.random.choice(persons)
    reason = np.random.choice(report_reasons)
    reporter.add_report(i, reportee, reason)
    reports.append(reporter.get_reports()[-1])

for p in persons:
    p.add_swipes(persons)
    swipes += p.get_swipes()


In [13]:
#Inserting real data for Profile

for person in persons:
  d = person.get_properties()
  c.execute("INSERT INTO Profile VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
            (d['pid'], 
             d['name'],
             d['sex'],
             d['age'],
             d['status'],
             d['height'],
             d['body_type'],
             d['diet'],
             d['drinks'],
             d['drugs'],
             d['smokes'],
             d['ethnicity'],
             d['education'],
             d['job'],
             d['location'],
             d['pets'],
             d['offspring'],
             d['religion'],
             d['speaks'],
             d['relationship_type'],
             d['signup_time'])
  )

  connection.commit() 

In [14]:
#Populating synthetic data of the Preference table

for preference in preferences:
  d = preference.get_preference()
  c.execute("INSERT INTO Preference VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
            (preference.get_owner().get_properties()['pid'], 
             d['sexual_orientation'],
             d['age'],
             d['height'],
             d['body_type'],
             d['diet'],
             d['drinks'],
             d['drugs'],
             d['smoke'],
             d['ethnicity'],
             d['education'],
             d['job'],
             d['location'],
             d['pet'],
             d['children'],
             d['religion'],
             d['languages'],
             d['relationship_type'])
  )

  connection.commit() 

In [15]:
#Populating synthetic data of the Report table

for report in reports:
  d = report
  c.execute("INSERT INTO Report VALUES (?, ?, ?, ?, ?)", 
            (d.rid, 
             d.owner.get_properties()['pid'],
             d.reportee.get_properties()['pid'],
             d.time,
             d.reason)
  )
  connection.commit() 

In [16]:
#Populating synthetic data of the Membership_order table

for membership in memberships:
  d = membership
  c.execute("INSERT INTO Membership_order(user_id,order_date,membership_duration) VALUES (?, ?, ?)", 
            (d.uid,
             d.order_date,
             d.duration)
  )

  connection.commit() 

In [17]:
#Populating synthetic data of the Swipe table

for swipe in swipes:
  d = swipe
  c.execute("INSERT INTO Swipe VALUES (?, ?, ?, ?)", 
            (d.owner.get_properties()['pid'], 
             d.target.get_properties()['pid'],
             d.direction,
             d.time)
  )
  connection.commit() 


In [18]:
#Populating synthetic data of the Message table

#According to the report section 2.1 Mini-world description, only the matched two can message each other
#We try to mimic and follow this rule in synthetic data generation
#We read from the existing match table and limit the two receivers/senders to the matched two users

c.execute("SELECT * FROM Match")
result = c.fetchall()

for match_id, uid1, uid2, start in result:
    start = datetime.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    end = datetime.datetime(2023, 12, 31)
    reps = np.random.randint(0, 15)
    for i in range(reps):
        rd = str(random_date(start, end))
        text = np.random.choice(texts)
        if i % 2 == 0:
            message = Message(match_id, uid1, uid2, text, rd)
        else:
            message = Message(match_id, uid2, uid1, text, rd)
        messages.append(message)  
        
for message in messages:
  d = message
  c.execute("INSERT INTO Message(match_id,sender_id,receiver_id,text,time) VALUES (?, ?, ?, ?, ?)", 
            (d.ma_id,
             d.u1_id,
             d.u2_id,
             d.text,
             d.time)
  )

  connection.commit() 

In [19]:
#Populating synthetic data of the Block table

#According to the report section 2.1 Mini-world description
#only the matched two can message each other, and therefore block each other from sending messages
#We try to mimic and follow this rule in synthetic data generation
#We read from the existing match table and limit the blocking and blocked users to the matched two users

for i, p in enumerate(result):
    if i % 10 == 0:
        match_id, uid1, uid2, start = p
        start = datetime.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
        end = datetime.datetime(2023, 12, 31)
        rd = str(random_date(start, end))
        block = Block(uid1, uid2, rd)
        blocks.append(block) 
        
for block in blocks:
  d = block
  c.execute("INSERT INTO Block(blocking_id,blocked_id,time) VALUES (?, ?, ?)", 
            (d.bing_id,
             d.bed_id,
             d.time)
  )

  connection.commit() 